<a href="https://colab.research.google.com/github/joostnjns/Google-Earth-Engine/blob/master/SVM_pre2000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
var cost_svm = 70;

// Query satellite image within bounding box
var aoi = ee.Geometry.Rectangle([14.35, 26.95, 14.51, 27.12]);
var collection = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1995-01-01', '1995-01-30').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);

// Get image of different years with least amount of cloud cover
var image_ = collection.sort('CLOUD_COVER').first(); // Main image of 95 to perform training

var collection86 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1985-12-01', '1986-01-30').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);
var image86 = collection86.sort('CLOUD_COVER').first();

var collection90 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1990-01-01', '1990-01-30').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);
var image90 = collection90.sort('CLOUD_COVER').first();

var collection95 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1995-01-01', '1995-01-30').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);
var image95 = collection95.sort('CLOUD_COVER').first();

var collection00 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('2000-01-01', '2000-02-10').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);
var image00 = collection00.sort('CLOUD_COVER').first();

var collection05 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('2005-01-01', '2005-02-20').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);
var image05 = collection05.sort('CLOUD_COVER').first();

var collection10 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('2010-01-01', '2010-01-30').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);
var image10 = collection10.sort('CLOUD_COVER').first();

// Create feature collection of all images
var collection = ee.ImageCollection.fromImages([image_,image86,image90,image95,image00,image05,image10])

function calcMSAVI2(image){
  var msavi2 = image.expression(
  '((2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2)*1000',  //to be in same scale as spectral bands 
  {
    'NIR': image.select('B4'), 
    'RED': image.select('B3')
  }
  );
  image = image.addBands([msavi2]);
  image = image.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','constant']).rename(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','msavi2']);
  return image
}

var image86 = calcMSAVI2(image86);
var image90 = calcMSAVI2(image90);
var image95 = calcMSAVI2(image95);
var image00 = calcMSAVI2(image00);
var image05 = calcMSAVI2(image05);
var image10 = calcMSAVI2(image10);

// Correct image10: all bands -500 except B6
//var corr = image10.select(['B1', 'B2', 'B3'], ['nB1', 'nB2', 'nB3'] ).subtract(500);
//image10 = image10.addBands(corr)

//var corr2 = image10.select(['B4', 'B5', 'B7'], ['nB4', 'nB5', 'nB7'] ).subtract(700);
//image10 = image10.addBands(corr2)

//image10 = image10.select(['nB1', 'nB2', 'nB3', 'nB4', 'nB5', 'B6', 'nB7', 'msavi2'],['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'msavi2'] )


// Correct image00: B1, B2, B3, B4 +400
//var corr = image00.select(['B1', 'B2', 'B3', 'B4'], ['nB1', 'nB2', 'nB3', 'nB4'] ).add(400);
//image00 = image00.addBands(corr)

//image00 = image00.select(['nB1', 'nB2', 'nB3', 'nB4', 'B5', 'B6', 'B7', 'msavi2'],['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'msavi2'] )

// Visualize satellite images
Map.setCenter(14.42, 27.04, 13);


// Define the visualization parameters.
var vizParams1 = {
  bands: ['B3','B2','B1'],
  min: 0,
  max: 5000,
  gamma: [0.9,1.1,1]
};

// Visualize satellite image              
Map.setCenter(14.42, 27.04, 13);
//Map.addLayer(image86.clip(aoi), vizParams1, 'true86');
//Map.addLayer(image90.clip(aoi), vizParams1, 'true90');
Map.addLayer(image95.clip(aoi), vizParams1, 'true95');
//Map.addLayer(image00.clip(aoi), vizParams1, 'true00');
Map.addLayer(image05.clip(aoi), vizParams1, 'true05');
Map.addLayer(image10.clip(aoi), vizParams1, 'true10');


//////////////////
//   TRAINING   //
//////////////////

// Load training data and sample ground truth polygons
var trainData = ee.FeatureCollection('users/jneujens/trainData');    // Load training data
//Map.addLayer(trainData);

var label = 'classname';     // Property containing land cover labels
var groundTrue = image95.sampleRegions({     // Get training from polygons on image
  collection: trainData,
  properties: ['classvalue'],
  scale: 30
});

// Split ground truth into training and test data
var withRandom = groundTrue.randomColumn('random');
var split = 0.99;  // Roughly 70% training, 30% testing.
var training = withRandom.filter(ee.Filter.lt('random', split));
var testing = withRandom.filter(ee.Filter.gte('random', split));

// Create an SVM classifier with custom parameters.
var classifier = ee.Classifier.svm({
  decisionProcedure: 'Voting',
  svmType: 'C_SVC',
  kernelType: 'LINEAR',
  cost: cost_svm
});

// Train the classifier.
var trained = classifier.train(training, 'classvalue', ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'msavi2']);

var results = collection.map(function(image){
  return image.clip(aoi).classify(trained);
})

// Display results
var sebha86 = image86.clip(aoi).classify(trained);
var sebha90 = image90.clip(aoi).classify(trained);
var sebha95 = image95.clip(aoi).classify(trained);
var sebha00 = image00.clip(aoi).classify(trained);
var sebha05 = image05.clip(aoi).classify(trained);
var sebha10 = image10.clip(aoi).classify(trained);

var vizParams2 = {min: 0, max: 2, palette: ['grey', 'green','red']}

// Display result
//Map.addLayer(sebha86, vizParams2, 'class86');
//Map.addLayer(sebha90, vizParams2, 'class90');
Map.addLayer(sebha95, vizParams2, 'class95');
//Map.addLayer(sebha00, vizParams2, 'class00');
Map.addLayer(sebha05, vizParams2, 'class05');
//Map.addLayer(sebha10, vizParams2, 'class10');             

